# HDBSCAN

In [ ]:
!pip3 install hdbscan

In [ ]:
import mouse.utils.constants as const
from mouse.utils import data_util
from mouse.utils import sound_util
import matplotlib.pyplot as plt
import numpy as np
import torch
import hdbscan
from scipy import ndimage
from tqdm.notebook import tqdm
from time import time

In [ ]:
data_folder: data_util.DataFolder = data_util.load_data([const.DATA_VPA])[0]
    
l = data_folder.signals[0].signal.shape[0]
squeak_signal = data_folder.signals[0].signal[int(l*3.8/120):int(l*5./120)]

%matplotlib inline
spec = sound_util.spectrogram(squeak_signal)
f, t, spec = spec.freqs, spec.times, spec.spec
spec = spec[f>18000,:]
plt.imshow(np.log(spec))
plt.show()
spec.shape

In [ ]:
def add_coords(spec):
    y = np.vstack([np.arange(spec.shape[0])]*spec.shape[1]).T
    x = np.vstack([np.arange(spec.shape[1])]*spec.shape[0])
    return np.dstack([spec, x, y])
    
add_coords(spec)[:2,:2, :].reshape(-1, 3)

In [ ]:
points = add_coords(spec).reshape(-1, 3)
np.max(points, axis=0), np.min(points[:, 0]), np.median(points[:, 0]), np.mean(points[:, 0])

In [ ]:
mult = 10
X = np.copy(points)
# X[:, 0] = np.log(X[:, 0])
# X[:, 0] = ndimage.median_filter(X[:, 0], 5)
X[:, 0] *= mult

clusterer = hdbscan.HDBSCAN(algorithm='best', alpha=1.0, approx_min_span_tree=True,
    gen_min_span_tree=False, leaf_size=40,
    metric='euclidean', min_cluster_size=40, min_samples=None, p=None)
t = time()
clusterer.fit(X)
X[:, 0] /= mult
print(time() - t, "[s]")

clusterer.labels_

In [ ]:
res = np.zeros(spec.shape, dtype=np.int64)
for i in range(spec.shape[1]):
    for j in range(spec.shape[0]):
        res[j, i] = clusterer.labels_[i+1172*j]
plt.imshow(res)
plt.show()
set(res.ravel())

In [ ]:
for i in set(clusterer.labels_):
    print("label", i, " | count", np.sum(i==res))
    plt.imshow(res==i)
    plt.show()

Conclusion: hdbscan could work, but it is slow. Other methods are better